In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Web
import Data.LinearMap.HerMetric
import Data.VectorSpace
import Data.Semigroup
import qualified Data.Foldable as Hask
import Control.Lens

import Control.Category.Constrained.Prelude
:opt no-lint  -- lint gives bogus warnings with constrained-categories

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [2]:
import Graphics.Dynamic.Plot.R2

type X = ℝ
type Y = ℝ

viewRange = plot [forceXRange (-2,4), forceYRange (-1,3)]

In [7]:
μ :: Linear ℝ Y (LocalLinear X Y)
μ = denseLinear $ \y -> denseLinear $ \δx -> y*δx

deq :: DifferentialEqn X Y
deq (Shade (_,y) δxy) = Shade' (μ $ y)
                        $ recipMetric (projector' (μ $ snd (fromDualWith δxy (0,1))))

In [21]:
itWhileJ :: (a -> Option a) -> a -> [a]
itWhileJ f x | Option (Just y) <- f x  = x : itWhileJ f y
itWhileJ _ x = [x]

skipN :: Int -> [a] -> [a]
skipN n xs = case splitAt n xs of
    ([], []) -> []
    ([x], []) -> [x]
    (xs₀, []) -> [head xs₀, last xs₀]
    (x:_, xs') -> x : skipN n xs'

In [29]:
tf :: PointsWeb X (Shade' Y)
tf = fromWebNodes euclideanMetric
        [(x, 1|±|[exp x*2-1.95]) | x<-[0, 0.05 .. 1] ]

tfs = itWhileJ (filterDEqnSolution_static deq) tf
plotWindow $ plot <$> reverse tfs
-- Hask.toList . localFocusWeb $ head tfs
-- onlyLeaves tf'

GraphWindowSpecR2{lBound=-0.16666666666666674, rBound=1.166666666666667, bBound=0.7199724780829753, tBound=2.9574487411053374, xResolution=640, yResolution=480}

![filtering the exponential function as a solution to an ODE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-exponential.png)

In [6]:
blockm ::  (X, Shade' Y)
blockm = (1, 2|±|[2])
blocks :: [(X, Shade' Y)]
blocks = [(x, 1|±|[1]) | x<-[0,2]]

Option (Just (shyl')) = filterDEqnSolution_loc deq (blockm, take 1 blocks)

[mcover] = pointsCovers [ (xs, ys')
                        | (xs, Shade' ys yse) <- blockm : tail blocks
                        , δy <- eigenCoSpan' yse
                        , ys' <- [ys.-~^δy, ys.+~^δy]] :: [Shade (X,Y)]
-- plotWindow [plot mcover]
mcover
Shade (mcxc, mcyc) mce = mcover
deq mcover
Shade' jc jn = deq mcover
mcyc ^-^ (jc $ 1)
applyLinMapMetric jn (toDualWith)

Shade {_shadeCtr = (1.5,1.5), _shadeExpanse = projector' (-0.23183461111316336,2.8933254794642274) ^+^ projector' (-0.7558126177102419,-6.0561290302438765e-2)}

Shade' {_shade'Ctr = outerProducts [(0.375,1.0)], _shade'Narrowness = projector (outerProducts [(0.47761194029850745,1.0)])}

1.125